In [64]:

import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    PromptTemplate,
)
from langchain.llms import OpenAI
import numpy as np




In [65]:
GENERATIONS_PATH = "../datasets/error_analysis/baseline_zero_shot_gpt3_50_50_merged.csv"


data = pd.read_csv(GENERATIONS_PATH)
data.head()

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation,structured_explanation_v2
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."


In [66]:
list = []
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
chain = LLMChain(llm=llm, prompt=PromptTemplate(
    input_variables=["error", "json"],
    template="""Please fix the JSON:
Error: {error}
JSON: {json}"""
)
)

for i, row in enumerate(data["structured_explanation_v2"]):
    try:
        row_json = json.loads(row)
        list.append(row_json)
    except Exception as e1:
        try:
            fixed = row.replace(",\n"+ row.split("\n")[-1], "]")
            row_json = json.loads(fixed)
            list.append(row_json)
            print("fixed:" + str(i))
        except Exception as e2:
            print("-----")
            print(e2)
            print(row)
            print(i)
            print("-----")
            # fixed_by_LLM = chain.run(error=e1, json=row)
            # try:
            #     row_json = json.loads(fixed_by_LLM)
            #     list.append(row_json)
            #     # print("It worked: " + fixed_by_LLM)
            # except Exception as e:
            #     print(e)
            #     print(i)
            #     print(fixed_by_LLM)
            #     print("------------------")
            #     print(row)
        


fixed:10
fixed:20
fixed:45
fixed:61
fixed:85
fixed:131
fixed:133
fixed:134
fixed:182


In [67]:
len(list)

221

In [72]:

# Create empty dictionaries to store statistics
attribute_counts = {}
attribute_total_importance = {}
attribute_importances = {}
# Iterate through the list of JSONs
for item in list:
    for item2 in item:
        attribute = item2['attribute']
        importance = float(item2['importance'])
        values = item2['values']

        # Update the dictionaries
        if attribute in attribute_counts:
            attribute_counts[attribute] += 1
            attribute_total_importance[attribute] += importance
            attribute_importances[attribute].append(importance)
        else:
            attribute_counts[attribute] = 1
            attribute_total_importance[attribute] = importance
            attribute_importances[attribute] = [importance]
 
# Calculate average and variance
attribute_average_importance = {attribute: total_importance / attribute_counts[attribute] for attribute, total_importance in attribute_total_importance.items()}
attribute_variance_importance = {attribute: np.var(importances) for attribute, importances in attribute_importances.items()}



# Create a DataFrame from the calculated statistics
attribute_stats = pd.DataFrame({
    'attribute': attribute_counts.keys(),
    'count': attribute_counts.values(),
    'average importance': [attribute_average_importance[attr] for attr in attribute_counts.keys()],
    'variance of importance': [attribute_variance_importance[attr] for attr in attribute_counts.keys()]
})

# Display the resulting table
attribute_stats

,attribute,count,average importance,variance of importance
0,brand,225,0.195556,0.035714
1,model,220,0.738409,0.080627
2,color,187,0.055615,0.016800
3,size,6,0.250000,0.089167
4,casing,4,0.450000,0.032500
...,...,...,...,...
154,interfaces,1,0.100000,0.000000
155,application compatibility,1,0.150000,0.000000
156,paper handling,1,0.200000,0.000000
157,cost efficiency,1,0.050000,0.000000
